In [1]:
%reset -f
import numpy
import theano
import theano.tensor as tt

# -----------------------------------------------------------------------------
#                               PYTHON LIBRARIES
# -----------------------------------------------------------------------------
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
dirToInclude=parentdir +'/features/'

sys.path.insert(0,dirToInclude)

import IeegConsts
from IeegConsts import *
from IeegFeatures import *

import pandas
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.cross_validation import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=4, threshold=10000, linewidth=100, edgeitems=999, suppress=True)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 100)
pd.set_option('expand_frame_repr', False)
pd.set_option('precision', 6)
   

train_dir=TRAIN_DATA_FOLDER_IN_ALL
ieegFeatures= IeegFeatures(train_dir, True)
df_cols_train=ieegFeatures.ieegGenCols()
print len(df_cols_train)
F_NAME_TRAIN= TRAIN_FEAT_BASE + TRAIN_PREFIX_ALL +'-feat_TRAIN_df.csv'
X_df_train=pandas.read_csv(F_NAME_TRAIN, engine='python') 
X_df_train.drop('Unnamed: 0', axis=1, inplace=True)
X_df_train.head(3)
X_df_train_SINGLE=X_df_train

# Numeric
import numpy as np
answers_1_SINGLE = list (X_df_train_SINGLE['result'].values)
answers_1_SINGLE = map(int, answers_1_SINGLE)
answers_1_SINGLE = np.asarray(answers_1_SINGLE)

X_df_train_SINGLE = X_df_train_SINGLE.drop('result', axis=1)
X_df_train_SINGLE.drop('id', axis=1, inplace=True)
X_df_train_SINGLE.drop('file', axis=1, inplace=True)
X_df_train_SINGLE.drop('file_size', axis=1, inplace=True)
X_df_train_SINGLE.drop('sequence_id', axis=1, inplace=True)
X_df_train_SINGLE.drop('patient_id', axis=1, inplace=True)
X_df_train_SINGLE.drop('segment', axis=1, inplace=True)
X_df_train_SINGLE=X_df_train_SINGLE.apply(lambda x: pandas.to_numeric(x, errors='ignore'))

X_df_train_SINGLE.head(3)


N = X_df_train_SINGLE.shape[0]
feats = X_df_train_SINGLE.shape[1]

X_df_train_SINGLE= X_df_train_SINGLE.as_matrix()

print X_df_train_SINGLE.shape

theano.config.floatX = 'float32'
rng = numpy.random

#D1 = (rng.randn(N, feats).astype(theano.config.floatX),rng.randint(size=N, low=0, high=2).astype(theano.config.floatX))

D1 = (X_df_train_SINGLE.astype(theano.config.floatX),answers_1_SINGLE.astype(theano.config.floatX))

Starting:ieegFeatures:2016-10-01 19:37:26.354359
Cols:775
775
(5971, 768)


In [2]:
# Declare Theano symbolic variables
#theano-cache purge
x = tt.matrix("x")
y = tt.vector("y")
w = theano.shared(rng.randn(feats).astype(theano.config.floatX), name="w")
b = theano.shared(numpy.asarray(0., dtype=theano.config.floatX), name="b")
x.tag.test_value = D1[0]
y.tag.test_value = D1[1]
#print "Initial model:"
#print w.get_value(), b.get_value()

# Construct Theano expression graph
p_1 = 1 / (1 + tt.exp(-tt.dot(x, w) - b))  # Probability of having a one
prediction = p_1  # The prediction that is done: 0 or 1
xent = -y * tt.log(p_1) - (1 - y) * tt.log(1 - p_1)  # Cross-entropy
cost = tt.cast(xent.mean(), 'float32') + \
       0.01 * (w ** 2).sum()  # The cost to optimize
gw, gb = tt.grad(cost, [w, b])

# Compile expressions to functions
train = theano.function(
            inputs=[x, y],
            outputs=[prediction, xent],
            updates={w: w - 0.01 * gw, b: b - 0.01 * gb},
            name="train")
predict = theano.function(inputs=[x], outputs=prediction,
            name="predict")

if any([x.op.__class__.__name__ in ['Gemv', 'CGemv', 'Gemm', 'CGemm'] for x in
train.maker.fgraph.toposort()]):
    print('Used the cpu')
elif any([x.op.__class__.__name__ in ['GpuGemm', 'GpuGemv'] for x in
train.maker.fgraph.toposort()]):
    print('Used the gpu')
else:
    print('ERROR, not able to tell if theano used the cpu or the gpu')
    print(train.maker.fgraph.toposort())

Used the cpu


In [3]:
training_steps = 50000

for i in range(training_steps):
    pred, err = train(D1[0], D1[1])
    
test_dir=TEST_DATA_FOLDER_IN_ALL
ieegFeatures= IeegFeatures(test_dir, False)
df_cols_test=ieegFeatures.ieegGenCols()
print len(df_cols_test)
F_NAME_TEST= TEST_FEAT_BASE + TEST_PREFIX_ALL +'-feat_TEST_df.csv'
X_df_TEST=pandas.read_csv(F_NAME_TEST, engine='python') 
X_df_TEST.drop('Unnamed: 0', axis=1, inplace=True)
# X_df_TEST.drop('id', axis=1, inplace=True)
X_df_TEST.drop('file', axis=1, inplace=True)
X_df_TEST.drop('patient_id', axis=1, inplace=True)
# X_df_TEST.drop('file_size', axis=1, inplace=True)
# X_df_TEST.drop('sequence_id', axis=1, inplace=True)
X_df_TEST.head(3)
# print X_df_TEST.shape

Starting:ieegFeatures:2016-10-01 19:40:41.035750
Cols:771
771


,id,mean_1,mean_2,mean_3,mean_4,mean_5,mean_6,mean_7,mean_8,mean_9,mean_10,mean_11,mean_12,mean_13,mean_14,mean_15,mean_16,median_1,median_2,median_3,median_4,median_5,median_6,median_7,median_8,median_9,median_10,median_11,median_12,median_13,median_14,median_15,median_16,std_1,std_2,std_3,std_4,std_5,std_6,std_7,std_8,std_9,std_10,std_11,std_12,std_13,std_14,std_15,std_16,skew_1,skew_2,skew_3,skew_4,skew_5,skew_6,skew_7,skew_8,skew_9,skew_10,skew_11,skew_12,skew_13,skew_14,skew_15,skew_16,kurt_1,kurt_2,kurt_3,kurt_4,kurt_5,kurt_6,kurt_7,kurt_8,kurt_9,kurt_10,kurt_11,kurt_12,kurt_13,kurt_14,kurt_15,kurt_16,min_1,min_2,min_3,min_4,min_5,min_6,min_7,min_8,min_9,min_10,min_11,min_12,min_13,min_14,min_15,min_16,max_1,max_2,max_3,max_4,max_5,max_6,max_7,max_8,max_9,max_10,max_11,max_12,max_13,max_14,max_15,max_16,psd_1,psd_2,psd_3,psd_4,psd_5,psd_6,psd_7,psd_8,psd_9,psd_10,psd_11,psd_12,psd_13,psd_14,psd_15,psd_16,psd_17,psd_18,psd_19,psd_20,psd_21,psd_22,psd_23,psd_24,psd_25,psd_26,psd_27,psd_28,psd_29,psd_30,psd_31,psd_32,psd_33,psd_34,psd_35,psd_36,psd_37,psd_38,psd_39,psd_40,psd_41,psd_42,psd_43,psd_44,psd_45,psd_46,psd_47,psd_48,psd_49,psd_50,psd_51,psd_52,psd_53,psd_54,psd_55,psd_56,psd_57,psd_58,psd_59,psd_60,psd_61,psd_62,psd_63,psd_64,psd_65,psd_66,psd_67,psd_68,psd_69,psd_70,psd_71,psd_72,psd_73,psd_74,psd_75,psd_76,psd_77,psd_78,psd_79,psd_80,psd_81,psd_82,psd_83,psd_84,psd_85,psd_86,psd_87,psd_88,psd_89,psd_90,psd_91,psd_92,psd_93,psd_94,psd_95,psd_96,psd_97,psd_98,psd_99,psd_100,psd_101,psd_102,psd_103,psd_104,psd_105,psd_106,psd_107,psd_108,psd_109,psd_110,psd_111,psd_112,psd_113,psd_114,psd_115,psd_116,psd_117,psd_118,psd_119,psd_120,psd_121,psd_122,psd_123,psd_124,psd_125,psd_126,psd_127,psd_128,psd_129,psd_130,psd_131,psd_132,psd_133,psd_134,psd_135,psd_136,psd_137,psd_138,psd_139,psd_140,psd_141,psd_142,psd_143,psd_144,psd_145,psd_146,psd_147,psd_148,psd_149,psd_150,psd_151,psd_152,psd_153,psd_154,psd_155,psd_156,psd_157,psd_158,psd_159,psd_160,AR_1,AR_2,AR_3,AR_4,AR_5,AR_6,AR_7,AR_8,AR_9,AR_10,AR_11,AR_12,AR_13,AR_14,AR_15,AR_16,AR_17,AR_18,AR_19,AR_20,AR_21,AR_22,AR_23,AR_24,AR_25,AR_26,AR_27,AR_28,AR_29,AR_30,AR_31,AR_32,AR_33,AR_34,AR_35,AR_36,AR_37,AR_38,AR_39,AR_40,AR_41,AR_42,AR_43,AR_44,AR_45,AR_46,AR_47,AR_48,AR_49,AR_50,AR_51,AR_52,AR_53,AR_54,AR_55,AR_56,AR_57,AR_58,AR_59,AR_60,AR_61,AR_62,AR_63,AR_64,AR_65,AR_66,AR_67,AR_68,AR_69,AR_70,AR_71,AR_72,AR_73,AR_74,AR_75,AR_76,AR_77,AR_78,AR_79,AR_80,fft_1,fft_2,fft_3,fft_4,fft_5,fft_6,fft_7,fft_8,fft_9,fft_10,fft_11,fft_12,fft_13,fft_14,fft_15,fft_16,fft_17,fft_18,fft_19,fft_20,fft_21,fft_22,fft_23,fft_24,fft_25,fft_26,fft_27,fft_28,fft_29,fft_30,fft_31,fft_32,fft_33,fft_34,fft_35,fft_36,fft_37,fft_38,fft_39,fft_40,fft_41,fft_42,fft_43,fft_44,fft_45,fft_46,fft_47,fft_48,fft_49,fft_50,fft_51,fft_52,fft_53,fft_54,fft_55,fft_56,fft_57,fft_58,fft_59,fft_60,fft_61,fft_62,fft_63,fft_64,fft_65,fft_66,fft_67,fft_68,fft_69,fft_70,fft_71,fft_72,fft_73,fft_74,fft_75,fft_76,fft_77,fft_78,fft_79,fft_80,fft_81,fft_82,fft_83,fft_84,fft_85,fft_86,fft_87,fft_88,fft_89,fft_90,fft_91,fft_92,fft_93,fft_94,fft_95,fft_96,fft_97,fft_98,fft_99,fft_100,fft_101,fft_102,fft_103,fft_104,fft_105,fft_106,fft_107,fft_108,fft_109,fft_110,fft_111,fft_112,fft_113,fft_114,fft_115,fft_116,fft_117,fft_118,fft_119,fft_120,fft_121,fft_122,fft_123,fft_124,fft_125,fft_126,fft_127,fft_128,fft_129,fft_130,fft_131,fft_132,fft_133,fft_134,fft_135,fft_136,fft_137,fft_138,fft_139,fft_140,fft_141,fft_142,fft_143,fft_144,fft_145,fft_146,fft_147,fft_148,fft_149,fft_150,fft_151,fft_152,fft_153,fft_154,fft_155,fft_156,fft_157,fft_158,fft_159,fft_160,fft_161,fft_162,fft_163,fft_164,fft_165,fft_166,fft_167,fft_168,fft_169,fft_170,fft_171,fft_172,fft_173,fft_174,fft_175,fft_176,fft_177,fft_178,fft_179,fft_180,fft_181,fft_182,fft_183,fft_184,fft_185,fft_186,fft_187,fft_188,fft_189,fft_190,fft_191,fft_192,fft_193,fft_194,fft_195,fft_196,fft_197,fft_198,fft_199,fft_200,fft_201,fft_202,fft_203,fft_204,fft_205,fft_

In [12]:
#------------------------------------------------------------------------------#
now = datetime.now()
import dis
sub_file = 'submission' + '_mcmc_all_'  + str(len(df_cols_train)) + str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + '.csv'

r= pandas.DataFrame.from_csv('sample_submission.csv')
print('Writing submission: ', sub_file)
f = open(sub_file, 'w') # append mode
f.write('File,Class\n')
total = 0

for index, row in r.iterrows():            
    id_str= index     
    arr = id_str.split("_")
    patient = int(arr[0])        
    new_id= ieegFeatures.getIdFromFileName(id_str) 
#     print str(new_id)
    
    X_df_single_row=X_df_TEST.loc[X_df_TEST['id'] == new_id]
    X_df_single_row.drop('id', axis=1, inplace=True)
#     print X_df_single_row
#     X_df_single_row= X_df_single_row[glm_factor]        
#     X_df_single_row.drop('file', axis=1, inplace=True)
#     X_df_single_row.drop('patient_id', axis=1, inplace=True)                    
#     X_df_single_row = np.asarray(X_df_single_row)        
    X_df_single_row= X_df_single_row.as_matrix().astype(theano.config.floatX) # must be float64
    c_pred= predict (X_df_single_row)
    
#     c_pred= predict(tuple (X_df_single_row)[0])
#     str1 = id_str + ',' + str(predict (tuple (X_df_single_row)[0])) + '\n'          
    str1 = id_str + ',' + str(tuple(c_pred)[0]) + '\n'                
#     print str1
    f.write(str1)
    
f.close()

print('Done writing submission: ', sub_file)

('Writing submission: ', 'submission_mcmc_all_7752016-10-01-20-10.csv')
('Done writing submission: ', 'submission_mcmc_all_7752016-10-01-20-10.csv')
